## Manually check the cached activations

ImportError: cannot import name 'TruthfulQADataset' from 'datasets' (/home/dev/persistent-storage/conda-envs/amongus/lib/python3.10/site-packages/datasets/__init__.py)